# Tutorial 5 -- Human Detection 
In this tutorial, we will use multithreading to capture both color and depth images. We will then employ the YOLOv11 neural network architecture for human detection. To ensure real-time performance, we will utilize the TensorRT deep learning framework. For more details, visit the following website: https://docs.ultralytics.com/guides/nvidia-jetson/#convert-model-to-tensorrt-and-run-inference

In [ ]:
# I have uploaded two YOLO TensorRT models to the LEARN page: the YOLO11L FP16 version and the YOLO11N FP32 version. The first model, 'YOLO11n.engine', runs at a faster speed but has limited detection accuracy. The second model, 'yolo11l_half.engine', is the FP16 version, as indicated by 'half' in its name.
# If you need other versions, please refer to the following link:
# https://docs.ultralytics.com/modes/export/#arguments

# Below is the code I used to convert the YOLO11L FP16 model
# from ultralytics import YOLO
# model = YOLO("yolo11l.pt")  
# model.export(format="engine",half=True)  # FP16
# Note: Generating the 'yolo11l.engine' file may take a long time.

### Step 1 Run YOLOV11 on recorded video data

Before running the program, download the pre-trained detection model and the video file 'color_video.avi', then place them in the current folder.

In [ ]:
# import cv2
# from ultralytics import YOLO
# import ipywidgets.widgets as widgets
# from IPython.display import display

# #create widgets for the displaying of the image
# display_color = widgets.Image(format='jpeg', width='45%') 
# display_depth = widgets.Image(format='jpeg', width='45%')  
# layout=widgets.Layout(width='100%')

# sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
# display(sidebyside) #display the widget

# #Convert a NumPy array to JPEG-encoded data for display
# def bgr8_to_jpeg(value):
#     return bytes(cv2.imencode('.jpg',value)[1])

# # Load the YOLO model
# model = YOLO("yolo11l_half.engine")
# # model = YOLO("yolo11n.engine")

# # Open the video file
# video_path = "color_video.avi"
# cap = cv2.VideoCapture(video_path)

# # Set the video codec and save the processed video.
# fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 'mp4v' codec, suitable for MP4 files
# width, height = 672,376 #VGA resolution
# fps = 30
# color_file = cv2.VideoWriter('color_video_processed.avi', fourcc, fps, (width, height))

# # Loop through the video frames
# while cap.isOpened():
#     # Read a frame from the video
#     success, frame = cap.read()

#     if success:
#         t1 = cv2.getTickCount()
#         # Run YOLO inference on the frame
#         results = model(frame,verbose=False)

#         # Visualize the results on the frame
#         annotated_frame = results[0].plot()
#         color_file.write(annotated_frame)

#         framed = frame.copy()
#         #set a confidence threshold to filter out unconfident boxes
#         #https://docs.ultralytics.com/modes/predict/#boxes
#         conf_threshold = 0.5
#         for result in results:
#             #get the human subject
#             for i in range (len(result.boxes.cls)):
#                 if(result.boxes.cls[i] == 0):  #human subject
#                     # print(result.boxes.xywh[i])
#                     if (result.boxes.conf[i] > conf_threshold): #you
#                         # print()
#                         bbox = result.boxes.xyxy[i]
#                         # print(bbox)
#                         cv2.rectangle(framed, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
#                         # cv2.imwrite('human.jpg',color_img)
                
#         scale = 0.3
#         resized_image = cv2.resize(annotated_frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
#         resized_image2 = cv2.resize(framed, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
#         display_color.value = bgr8_to_jpeg(resized_image)
#         display_depth.value = bgr8_to_jpeg(resized_image2)
        
#         # total_time = (cv2.getTickCount() - t1) / cv2.getTickFrequency()
#     else:
#         # Break the loop if the end of the video is reached
#         break

# # Release the video capture object and close the display window
# cap.release()
# color_file.release()

### Step 2 Start the ZED2i Camera system

In [ ]:
# You will need to load the YOLO model if you skip the first code block.
from ultralytics import YOLO
model = YOLO("yolo11l_half.engine")

#Start the camera system
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.HD1080 #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())   
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data

### Step 3 Perform object detection on live video data

In [ ]:
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import time
import motors

# Distances from human
MIN_DISTANCE = 400.0
MAX_DISTANCE = 700.0
FRAME_CENTER = 336
TOO_CLOSE = "nan"

robot = motors.MotorsYukon(mecanum=False)

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget


# def get_motion_command(bbox, depth_value):
    
#     x_center = int((bbox[0] + bbox[2]) / 2)  # Bounding box center X
#     motion_command = ""

#     # Direction control
#     if x_center < FRAME_CENTER - 50:  # Adjust threshold as needed
#         motion_command += "LEFT "
#         robot.left(0.2)
#         time.sleep(0.5)
#     elif x_center > FRAME_CENTER + 50:
#         motion_command += "RIGHT "
#         robot.right(0.2)
#         time.sleep(0.5)
#     else:
#         motion_command += "STRAIGHT "
#         #robot.forward(0.1)
#         time.sleep(0.5)

#     # Speed control based on distance
#     if depth_value < MIN_DISTANCE:
#         motion_command += "STOP"
#         robot.stop()
#         time.sleep(0.5)
#     elif depth_value > MAX_DISTANCE:
#         motion_command += "FORWARD FAST"
#         robot.forward(0.7)
#         time.sleep(0.5)
#     else:
#         motion_command += "FORWARD SLOW"
#         robot.forward(0.04)
#         time.sleep(0.5)

#     return motion_command



def get_motion_command(bbox, depth_value):
    
    x_center = int((bbox[0] + bbox[2]) / 2)  # Bounding box center X
    motion_command = ""

    # Direction control
    if x_center < FRAME_CENTER - 70:  # Adjust threshold as needed
        motion_command += "LEFT "
        # robot.left(0.2)
        # time.sleep(0.1)
        if depth_value < MIN_DISTANCE:
            motion_command += "STOP"
            robot.left(0.2)
            time.sleep(0.1)
            robot.stop()
            # time.sleep(0.5)
        elif depth_value == TOO_CLOSE:
            robot.left(0.2)
            time.sleep(0.1)
            robot.stop()
        elif depth_value > MAX_DISTANCE:
            motion_command += "FORWARD FAST"
            # robot.left(0.5)
            robot.left(0.2)
            time.sleep(0.1)
            robot.forward(0.2)
            time.sleep(0.1)
        else:
            motion_command += "FORWARD SLOW"
            robot.left(0.2)
            time.sleep(0.1)
            #robot.left(0.5)
            robot.forward(0.04)
            time.sleep(0.1)


        
    elif x_center > FRAME_CENTER + 70:
        motion_command += "RIGHT "
        # robot.right(0.2)
        # time.sleep(0.1)
        if depth_value < MIN_DISTANCE:
            motion_command += "STOP"
            robot.right(0.2)
            time.sleep(0.1)
            robot.stop()
            # time.sleep(0.5)
        elif depth_value == TOO_CLOSE:
            robot.right(0.2)
            time.sleep(0.1)
            robot.stop()
        elif depth_value > MAX_DISTANCE:
            motion_command += "FORWARD FAST"
            robot.right(0.2)
            time.sleep(0.1)
            # robot.right(0.5)
            robot.forward(0.2)
            time.sleep(0.1)
        else:
            motion_command += "FORWARD SLOW"
            robot.right(0.2)
            time.sleep(0.1)
            #robot.right(0.5)
            robot.forward(0.04)
            time.sleep(0.1)


        
    else:
        motion_command += "STRAIGHT "
        # robot.forward(0.04)
        # time.sleep(0.1)
        if depth_value < MIN_DISTANCE:
            motion_command += "STOP"
            robot.stop()
            # time.sleep(0.5)
        elif depth_value == TOO_CLOSE:
            robot.stop()
        elif depth_value > MAX_DISTANCE:
            motion_command += "FORWARD FAST"
            robot.forward(0.3)
            time.sleep(0.1)
        else:
            motion_command += "FORWARD SLOW"
            robot.forward(0.04)
            time.sleep(0.1)

    # Speed control based on distance
    # if depth_value < MIN_DISTANCE:
    #     motion_command += "STOP"
    #     robot.stop()
    #     time.sleep(0.5)
    # elif depth_value > MAX_DISTANCE:
    #     motion_command += "FORWARD FAST"
    #     robot.forward(0.7)
    #     time.sleep(0.5)
    # else:
    #     motion_command += "FORWARD SLOW"
    #     robot.forward(0.04)
    #     time.sleep(0.5)

    time.sleep(0.1)

    return motion_command


    


#callback function, invoked when traitlets detects the changing of the color image
def func(change):

    frame = change['new']
    # t1 = cv2.getTickCount()
    results = model(frame,verbose=False)
    # total_time = (cv2.getTickCount() - t1) / cv2.getTickFrequency()
    # print('total_time ',total_time)

    conf_threshold = 0.5
    for result in results: #the input image is one so only one result avaialbe
        for i in range (len(result.boxes.cls)):
            if(result.boxes.cls[i] == 0):  #human subject
                if (result.boxes.conf[i] > conf_threshold): #you
                    bbox = result.boxes.xyxy[i]
                    depth_value = camera.depth.get_value(int((bbox[0] + bbox[2]) / 2), int((bbox[1] + bbox[3]) / 2))[1]
                    command = get_motion_command(bbox, depth_value)
                    print (command)
                    print("Depth Value: ", depth_value)
                    #Act on command recieved and move the robot




                    
                    cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
                    cv2.putText(frame, f"Distance: {depth_value:.2f}m", (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    
    

    #Scaling is necessary for real-time data display.
    scale = 0.1 
    resized_image = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_image)
    
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image , alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
    #cv2.imshow("Human Following: ", frame)

camera.observe(func, names=['color_value'])
# camera.stop()